<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object
# Realizar la solicitud HTTP GET a la página de Wikipedia
response = requests.get(static_url)

# Verificar que la solicitud fue exitosa
if response.status_code == 200:
    print("Request was successful!")
else:
    print(f"Request failed with status code: {response.status_code}")

Request was successful!


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content

soup = BeautifulSoup(response.text, 'html.parser')


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [7]:
# Use soup.title attribute
# Imprimir el título de la página para verificar la creación correcta del objeto BeautifulSoup
print(soup.title)

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# Utilizar find_all para encontrar todas las tablas en el documento HTML
html_tables = soup.find_all('table')

# Imprimir la cantidad de tablas encontradas para verificar
print(f"Number of tables found: {len(html_tables)}")

Number of tables found: 25


Starting from the third table is our target table contains the actual launch records.


You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [13]:
# Inicializar la lista para almacenar los nombres de las columnas
column_names = []

# Iterar sobre cada tabla encontrada
for table in html_tables:
    # Encontrar todos los elementos <th> en la tabla
    th_elements = table.find_all('th')
    
    # Aplicar la función extract_column_from_header() a cada <th> para extraer los nombres de las columnas
    for th in th_elements:
        column_name = extract_column_from_header(th)
        
        # Agregar el nombre de la columna a la lista si no está vacío
        if column_name and column_name not in column_names:
            column_names.append(column_name)

# Imprimir la lista completa de nombres de columnas para verificar
print("Extracted Column Names:", column_names)

Extracted column name: Flight No.
Extracted column name: Date and time ( )
Extracted column name: 
Extracted column name: Launch site
Extracted column name: Payload
Extracted column name: Payload mass
Extracted column name: Orbit
Extracted column name: Customer
Extracted column name: Launch outcome
Extracted column name: 
Extracted column name: Flight No.
Extracted column name: Date and time ( )
Extracted column name: 
Extracted column name: Launch site
Extracted column name: Payload
Extracted column name: Payload mass
Extracted column name: Orbit
Extracted column name: Customer
Extracted column name: Launch outcome
Extracted column name: 
Extracted column name: Flight No.
Extracted column name: Date and time ( )
Extracted column name: 
Extracted column name: Launch site
Extracted column name: Payload
Extracted column name: Payload mass
Extracted column name: Orbit
Extracted column name: Customer
Extracted column name: Launch outcome
Extracted column name: 
Extracted column name: Fligh

In [14]:
def extract_column_from_header(row):
    """
    Esta función devuelve el nombre de la columna desde una celda de tabla HTML.
    """
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
    
    # Extraer solo el texto de los elementos en 'row.contents'
    column_name = ' '.join([str(item) for item in row.contents if isinstance(item, str) or isinstance(item, str)])
    
    # Filtrar nombres de columna vacíos o solo dígitos
    if not column_name.strip().isdigit():
        column_name = column_name.strip()
        print("Extracted column name:", column_name)  # Imprimir el nombre de la columna aquí
        return column_name

Check the extracted column names


In [15]:
# Asumiendo que ya tienes el código para llenar la lista 'column_names'

# Imprimir la lista completa de nombres de columnas
print("Extracted Column Names:", column_names)

Extracted Column Names: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome', 'N/A', 'FH 2', 'FH 3', 'Flight', 'Demonstrations', 'logistics', 'Crewed', 'Commercial satellites', 'Scientific satellites', 'Military satellites', 'Rideshares', 'Transporter', 'Bandwagon', 'Current', 'In development', 'Retired', 'Cancelled', 'Spacecraft', 'Cargo', 'Test vehicles', 'Unflown', 'Orbital', 'Atmospheric', 'Landing sites', 'Other facilities', 'Support', 'Contracts', 'R&D programs', 'Key people', 'Related', 'General', 'People', 'Vehicles', 'Launches by rocket type', 'Launches by spaceport', 'Agencies, companies and facilities', 'Other mission lists and timelines']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [16]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [17]:
extracted_row = 0

# Extraer cada tabla
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Obtener las filas de la tabla
    for rows in table.find_all("tr"):
        # Verificar si el primer encabezado de la fila es un número correspondiente a un número de lanzamiento
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()
        else:
            flag = False

        # Obtener los elementos de la fila
        row = rows.find_all('td')

        # Si es un número, guardar las celdas en el diccionario
        if flag:
            extracted_row += 1

            # Flight Number value
            launch_dict['Flight No.'].append(flight_number)

            # Date value
            datatimelist = date_time(row[0])
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)

            # Time value
            time = datatimelist[1]
            launch_dict['Time'].append(time)

            # Booster version
            bv = booster_version(row[1])
            if not bv:
                bv = row[1].a.string if row[1].a else None
            launch_dict['Version Booster'].append(bv)

            # Launch Site
            launch_site = row[2].a.string if row[2].a else None
            launch_dict['Launch site'].append(launch_site)

            # Payload
            payload = row[3].a.string if row[3].a else None
            launch_dict['Payload'].append(payload)

            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)

            # Orbit
            orbit = row[5].a.string if row[5].a else None
            launch_dict['Orbit'].append(orbit)

            # Customer
            customer = row[6].a.string if row[6].a else None
            launch_dict['Customer'].append(customer)

            # Launch outcome
            launch_outcome = list(row[7].strings)[0] if row[7].strings else None
            launch_dict['Launch outcome'].append(launch_outcome)

            # Booster landing
            booster_landing = landing_status(row[8]) if row[8] else None
            launch_dict['Booster landing'].append(booster_landing)

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [18]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

/home/jupyterlab/conda/envs/python/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


In [19]:
import requests
import pandas as pd

# URL de la API de SpaceX para obtener los lanzamientos pasados
spacex_url = "https://api.spacexdata.com/v4/launches/past"

# Realizar la solicitud GET a la API de SpaceX
response = requests.get(spacex_url)

# Verificar que la solicitud fue exitosa
if response.status_code == 200:
    print("Request was successful!")
    # Convertir la respuesta a un DataFrame de pandas usando pd.json_normalize
    data = response.json()
    df_api = pd.json_normalize(data)
    
    # Verificar las columnas presentes en el DataFrame
    print("Columnas en el DataFrame:", df_api.columns)
    
    # Filtrar los lanzamientos de Falcon 9 eliminando los de Falcon 1
    falcon9_launches = df_api[df_api['rocket'] != 'Falcon 1']
    print("Número de lanzamientos de Falcon 9:", len(falcon9_launches))
    
    # Verificar valores faltantes en la columna landingPad
    if 'cores' in df_api.columns:
        # Si los datos de landingPad están en la columna 'cores', extraer la información
        df_api['landingPad'] = df_api['cores'].apply(lambda x: x[0]['landpad'] if isinstance(x, list) and len(x) > 0 and 'landpad' in x[0] else None)
        missing_landingPad = df_api['landingPad'].isna().sum()
        print("Valores faltantes en landingPad:", missing_landingPad)
    else:
        print("'landingPad' no está presente en el DataFrame.")
else:
    print("Request failed. Status code:", response.status_code)

Request was successful!
Columnas en el DataFrame: Index(['static_fire_date_utc', 'static_fire_date_unix', 'net', 'window',
       'rocket', 'success', 'failures', 'details', 'crew', 'ships', 'capsules',
       'payloads', 'launchpad', 'flight_number', 'name', 'date_utc',
       'date_unix', 'date_local', 'date_precision', 'upcoming', 'cores',
       'auto_update', 'tbd', 'launch_library_id', 'id', 'fairings.reused',
       'fairings.recovery_attempt', 'fairings.recovered', 'fairings.ships',
       'links.patch.small', 'links.patch.large', 'links.reddit.campaign',
       'links.reddit.launch', 'links.reddit.media', 'links.reddit.recovery',
       'links.flickr.small', 'links.flickr.original', 'links.presskit',
       'links.webcast', 'links.youtube_id', 'links.article', 'links.wikipedia',
       'fairings'],
      dtype='object')
Número de lanzamientos de Falcon 9: 187
Valores faltantes en landingPad: 36


In [21]:
import requests
import pandas as pd

# URL de la API de SpaceX para obtener los lanzamientos pasados
spacex_url = "https://api.spacexdata.com/v4/launches/past"

# Realizar la solicitud GET a la API de SpaceX
response = requests.get(spacex_url)

In [22]:
if response.status_code == 200:
    print("Request was successful!")
    # Convertir la respuesta a un DataFrame de pandas usando pd.json_normalize
    data = response.json()
    df_api = pd.json_normalize(data)
else:
    print("Request failed. Status code:", response.status_code)

Request was successful!


In [23]:
# Obtener los IDs únicos de los cohetes en los lanzamientos
rocket_ids = df_api['rocket'].unique()

# Diccionario para almacenar el nombre de los cohetes
rocket_names = {}

# Obtener el nombre de cada cohete a partir de su ID
for rocket_id in rocket_ids:
    rocket_response = requests.get(f"https://api.spacexdata.com/v4/rockets/{rocket_id}")
    if rocket_response.status_code == 200:
        rocket_data = rocket_response.json()
        rocket_names[rocket_id] = rocket_data['name']

# Crear una nueva columna en el DataFrame con los nombres de los cohetes
df_api['rocket_name'] = df_api['rocket'].map(rocket_names)

# Filtrar solo los lanzamientos de Falcon 9
falcon9_launches = df_api[df_api['rocket_name'] == 'Falcon 9']
print("Número de lanzamientos de Falcon 9:", len(falcon9_launches))

Número de lanzamientos de Falcon 9: 179


In [24]:
# Extraer la información del lugar de aterrizaje desde la columna cores
df_api['landingPad'] = df_api['cores'].apply(lambda x: x[0]['landpad'] if isinstance(x, list) and len(x) > 0 and x[0].get('landpad') else None)

# Contar los valores faltantes en landingPad
missing_landingPad = df_api['landingPad'].isnull().sum()
print("Valores faltantes en landingPad:", missing_landingPad)

Valores faltantes en landingPad: 36


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
